In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import glob
import re

################################################################
# 변수
################################################################
## CSV 폴더
sourceCsvFolder = '../datasets/**/*.csv'
## 결과 파일명
resultFileName = f'./MultiResults_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
## 쿼리 조건
conditions = [
    ['avg90',    0.05, 0.05],  # 상위  5%, 하위  5%를 제외한 90%의 일일 평균값
    ['avg90H10', 0.05, 0.85],  # 상위  5%, 하위 85%를 제외한 10%의 일일 평균값
    ['avg90L10', 0.85, 0.05],  # 상위 85%, 하위  5%를 제외한 10%의 일일 평균값
]

################################################################
# 소스코드
################################################################
def ReadSingleCSV(filename):
    df = pd.read_csv(filename,parse_dates=[0], na_values=[' '])
    # df에서 colums 추출하기
    col = list(map(lambda x:x.replace('\\','/'), df.columns))

    # 설비 명 찾기
    machineName = re.match('^//([^/]*)/', col[2]).group(1)

    # 첫번째 열 이름 바꾸기
    col[0] = "Date"

    # Column 내 장비명을 삭제하여 정규화하기
    col2 = map(lambda x: re.sub('^//([^/]*)/', '', x), col)

    # df의 Columns 이름 변경
    df.columns = col2

    #df에 machineName 컬럼 추가
    df["MachineName"] = machineName 
    df.set_index(["Date", "MachineName"], inplace=True)

    ret_cond = []
    for x in conditions:
        df90 = df.where(df <= df.quantile(q=(1-x[1]))).where(df >= df.quantile(q=x[2])).unstack().resample('1h').mean().interpolate()
        df90["Type"] = x[0]
        df90.set_index("Type", append=True, inplace=True )
        ret_cond.append(df90.unstack())
    
    # 결과 합치기
    ret = pd.concat(ret_cond)
    return ret







In [2]:
result = pd.DataFrame()

files = glob.glob(sourceCsvFolder, recursive=True)

files

['../datasets/WIN-3OPFVMF4N3A_20200205/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200206/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200207/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200208/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200209/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200210/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200211/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200213/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200214/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200215/DataCollector01.csv',
 '../datasets/WIN-3OPFVMF4N3A_20200216/DataCollector01.csv']

In [3]:
for idx, x in enumerate(files):
    try:
        df = ReadSingleCSV(x)
        result = pd.concat([df, result])
    except Exception as ex:
        print (f'Runtime Error: {ex}')
    print (f'{idx+1}/{len(files)} : {x} ')

1/11 : ../datasets/WIN-3OPFVMF4N3A_20200205/DataCollector01.csv 
2/11 : ../datasets/WIN-3OPFVMF4N3A_20200206/DataCollector01.csv 
3/11 : ../datasets/WIN-3OPFVMF4N3A_20200207/DataCollector01.csv 
4/11 : ../datasets/WIN-3OPFVMF4N3A_20200208/DataCollector01.csv 
5/11 : ../datasets/WIN-3OPFVMF4N3A_20200209/DataCollector01.csv 
6/11 : ../datasets/WIN-3OPFVMF4N3A_20200210/DataCollector01.csv 
7/11 : ../datasets/WIN-3OPFVMF4N3A_20200211/DataCollector01.csv 
8/11 : ../datasets/WIN-3OPFVMF4N3A_20200213/DataCollector01.csv 
9/11 : ../datasets/WIN-3OPFVMF4N3A_20200214/DataCollector01.csv 
10/11 : ../datasets/WIN-3OPFVMF4N3A_20200215/DataCollector01.csv 
11/11 : ../datasets/WIN-3OPFVMF4N3A_20200216/DataCollector01.csv 


In [12]:
lastret = result.resample(rule='1d', level=0).mean()

In [26]:
lastret1 = lastret.stack(level=1).stack(level=0)

In [65]:
lastret2 = lastret1.reset_index()

In [66]:
df2 = pd.DataFrame(columns=lastret2.columns)

In [67]:
total_memory = {"WIN-3OPFVMF4N3A": 8}

In [68]:
df2

Type,Date,MachineName,level_2,avg90,avg90H10,avg90L10


In [78]:
for index, row in lastret2.iterrows():
    if(row[2] == 'Memory/Available MBytes'):
        if row[1] in total_memory:
            newRow = [row[0], row[1], "Memory/Available Percent"]
            for x in conditions:
                newRow.append(row[x[0]] / (total_memory[row[1]] * 1024))
            print(newRow)
            newDf = pd.DataFrame([newRow], columns=lastret2.columns)
            lastret3=lastret2.append(newDf, ignore_index=True)
        

[Timestamp('2020-02-05 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.8483957703423229, 0.848839705107541, 0.847338945940369]
[Timestamp('2020-02-06 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.848352243604422, 0.8488378338334478, 0.8473270623564214]
[Timestamp('2020-02-07 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.8477216507885632, 0.8483470293490836, 0.8468017338609866]
[Timestamp('2020-02-08 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.8461669538159741, 0.8466231117611476, 0.845237596478743]
[Timestamp('2020-02-09 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.8466532835980546, 0.847147019161065, 0.8456728785453353]
[Timestamp('2020-02-10 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.8464188646400855, 0.8469131904521021, 0.8454548003986448]
[Timestamp('2020-02-11 00:00:00'), 'WIN-3OPFVMF4N3A', 'Memory/Available Percent', 0.8454427534516498, 0.845890185964668, 0.8444076462611261]
[Timestamp('

In [80]:
lastret2

Type,Date,MachineName,level_2,avg90,avg90H10,avg90L10
0,2020-02-05,WIN-3OPFVMF4N3A,Memory/% Committed Bytes In Use,1.732069e+01,1.741042e+01,1.728683e+01
1,2020-02-05,WIN-3OPFVMF4N3A,Memory/Available MBytes,6.950058e+03,6.953695e+03,6.941401e+03
2,2020-02-05,WIN-3OPFVMF4N3A,Memory/Cache Bytes,9.009619e+07,9.131839e+07,8.955987e+07
3,2020-02-05,WIN-3OPFVMF4N3A,Network Interface(6TO4 Adapter)/Bytes Total/sec,0.000000e+00,0.000000e+00,0.000000e+00
4,2020-02-05,WIN-3OPFVMF4N3A,Network Interface(6TO4 Adapter)/Output Queue L...,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
554,2020-02-17,WIN-3OPFVMF4N3A,Processor(0)/% User Time,0.000000e+00,0.000000e+00,0.000000e+00
555,2020-02-17,WIN-3OPFVMF4N3A,Processor(1)/% User Time,0.000000e+00,0.000000e+00,0.000000e+00
556,2020-02-17,WIN-3OPFVMF4N3A,Processor(2)/% User Time,0.000000e+00,3.130032e-01,0.000000e+00
557,2020-02-17,WIN-3OPFVMF4N3A,Processor(3)/% User Time,3.235993e-02,3.235993e-02,0.000000e+00


In [81]:
lastret3

Type,Date,MachineName,level_2,avg90,avg90H10,avg90L10
0,2020-02-05,WIN-3OPFVMF4N3A,Memory/% Committed Bytes In Use,1.732069e+01,1.741042e+01,1.728683e+01
1,2020-02-05,WIN-3OPFVMF4N3A,Memory/Available MBytes,6.950058e+03,6.953695e+03,6.941401e+03
2,2020-02-05,WIN-3OPFVMF4N3A,Memory/Cache Bytes,9.009619e+07,9.131839e+07,8.955987e+07
3,2020-02-05,WIN-3OPFVMF4N3A,Network Interface(6TO4 Adapter)/Bytes Total/sec,0.000000e+00,0.000000e+00,0.000000e+00
4,2020-02-05,WIN-3OPFVMF4N3A,Network Interface(6TO4 Adapter)/Output Queue L...,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
555,2020-02-17,WIN-3OPFVMF4N3A,Processor(1)/% User Time,0.000000e+00,0.000000e+00,0.000000e+00
556,2020-02-17,WIN-3OPFVMF4N3A,Processor(2)/% User Time,0.000000e+00,3.130032e-01,0.000000e+00
557,2020-02-17,WIN-3OPFVMF4N3A,Processor(3)/% User Time,3.235993e-02,3.235993e-02,0.000000e+00
558,2020-02-17,WIN-3OPFVMF4N3A,Processor(_Total)/% User Time,1.559798e-01,1.559798e-01,0.000000e+00


In [93]:
lastret4 = lastret3.join(lastret3["level_2"].str.split('/', 1, expand=True))

In [94]:
lastret4.columns

Index(['Date', 'MachineName', 'level_2', 'avg90', 'avg90H10', 'avg90L10', 0,
       1],
      dtype='object')

In [98]:
col = lastret3.columns.to_list()
col.insert(3, 1)
col.insert(3, 0)
col

['Date', 'MachineName', 'level_2', 0, 1, 'avg90', 'avg90H10', 'avg90L10']

In [99]:
lastret5 = lastret4[col]

In [100]:
lastret5

,Date,MachineName,level_2,0,1,avg90,avg90H10,avg90L10
0,2020-02-05,WIN-3OPFVMF4N3A,Memory/% Committed Bytes In Use,Memory,% Committed Bytes In Use,1.732069e+01,1.741042e+01,1.728683e+01
1,2020-02-05,WIN-3OPFVMF4N3A,Memory/Available MBytes,Memory,Available MBytes,6.950058e+03,6.953695e+03,6.941401e+03
2,2020-02-05,WIN-3OPFVMF4N3A,Memory/Cache Bytes,Memory,Cache Bytes,9.009619e+07,9.131839e+07,8.955987e+07
3,2020-02-05,WIN-3OPFVMF4N3A,Network Interface(6TO4 Adapter)/Bytes Total/sec,Network Interface(6TO4 Adapter),Bytes Total/sec,0.000000e+00,0.000000e+00,0.000000e+00
4,2020-02-05,WIN-3OPFVMF4N3A,Network Interface(6TO4 Adapter)/Output Queue L...,Network Interface(6TO4 Adapter),Output Queue Length,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...
555,2020-02-17,WIN-3OPFVMF4N3A,Processor(1)/% User Time,Processor(1),% User Time,0.000000e+00,0.000000e+00,0.000000e+00
556,2020-02-17,WIN-3OPFVMF4N3A,Processor(2)/% User Time,Processor(2),% User Time,0.000000e+00,3.130032e-01,0.000000e+00
557,2020-02-17,WIN-3OPFVMF4N3A,Processor(3)/% User Time,Processor(3),% User Time,3.235993e-02,3.235993e-02,0.000000e+00
558,2020-02-17,WIN-3OPFVMF4N3A,Processor(_Total)/% User Time,Processor(_Total),% User Time,1.559798e-01,1.559798e-01,0.000000e+00
